# Evalution and Analysis

## Imports:

In [1]:
from pathlib import Path

## Get predictions and metrics:

In [2]:
from Helper.evaluation_helpers import get_predictions, calc_metrics

In [3]:
predsPath= Path('../Predictions/')
predsDf=get_predictions(predsPath)
display(predsDf)

,ground_truth,ARDL,GRU,NHITS,Naive,NBEATSx,SARIMAX
observation_date,,,,,,,
01/2021,106.083,105.785040,104.798742,106.287415,105.69,105.880539,106.038947
02/2021,106.471,105.785040,104.614738,106.401459,105.69,105.993225,106.501714
03/2021,107.032,105.785040,104.392772,106.361130,105.69,106.098434,106.877229
04/2021,107.632,105.860227,104.176452,106.385925,105.69,106.191010,107.086592
05/2021,108.184,106.270458,103.988449,106.396194,105.69,106.291138,107.423990
06/2021,108.748,106.605603,103.839441,106.481056,105.69,106.396713,107.791156
07/2021,109.275,106.960277,103.721934,106.481674,105.69,106.486832,108.105803
08/2021,109.720,107.328459,103.626826,106.550713,105.69,106.595963,108.381933
09/2021,110.047,107.671517,103.544718,106.595749,105.69,106.687790,108.524800


# Calculate Metrics:

## Horizon = 1

In [4]:
# WARNING: r2 is not well defined for less than 2 samples
metrics_1= calc_metrics(predsDf, horizon=1).drop('r2', axis=1)
display(metrics_1)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Library/Frameworks/Pyth

,RMSE,MAE
ARDL,0.29796,0.29796
GRU,1.284258,1.284258
NHITS,0.204415,0.204415
Naive,0.393,0.393
NBEATSx,0.202461,0.202461
SARIMAX,0.044053,0.044053


## Horizon = 3

In [5]:
metrics_3= calc_metrics(predsDf, horizon=3)
display(metrics_3)

,RMSE,MAE,r2
ARDL,0.83949,0.743627,-3.643714
GRU,2.00504,1.926583,-25.489902
NHITS,0.406894,0.314942,-0.09093
Naive,0.92473,0.838667,-4.63461
NBEATSx,0.616659,0.537934,-1.505672
SARIMAX,0.094583,0.076513,0.941053


## Horizon = 6

In [6]:
metrics_6= calc_metrics(predsDf, horizon=6)
display(metrics_6)

,RMSE,MAE,r2
ARDL,1.500273,1.343099,-1.59305
GRU,3.309027,3.056568,-11.614523
NHITS,1.315591,1.040942,-0.993938
Naive,1.910852,1.668333,-3.206534
NBEATSx,1.433454,1.21649,-1.367213
SARIMAX,0.550375,0.4153,0.651031


## Horizon = 12

In [7]:
metrics_12= calc_metrics(predsDf, horizon=12)
display(metrics_12)

,RMSE,MAE,r2
ARDL,2.314447,2.095828,-0.503084
GRU,5.586933,5.067886,-7.758622
NHITS,3.104123,2.549541,-1.70375
Naive,3.786199,3.282,-3.022498
NBEATSx,2.968396,2.53351,-1.472479
SARIMAX,1.446428,1.142871,0.41294


# Analysis: